In [3]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [4]:
# Ensure GPU is being used
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU detected: {gpus}")
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
else:
    print("⚠️ No GPU detected, running on CPU...")

✅ GPU detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
# 1. Set some parameters
IMG_HEIGHT, IMG_WIDTH = 40, 40
BATCH_SIZE = 64
EPOCHS = 100  # with early stopping
NUM_CLASSES = 20

In [6]:
# 2. Load labels
labels_df = pd.read_csv('F:\InfTech\Prodotti\Python\Deep\data\labels.csv')  # <<== CHANGE path if needed
labels_df['label'] = labels_df['label'].astype(str)  # Keras expects labels as strings for categorical

# 3. Train/Validation split
train_df, val_df = train_test_split(labels_df, test_size=0.2, stratify=labels_df['label'], random_state=42)

# add 0 at the start if the id is less than 5 digits
train_df['id'] = train_df['id'].apply(lambda x: str(x).zfill(5))
val_df['id'] = val_df['id'].apply(lambda x: str(x).zfill(5))
# add .jpeg at the end of the id
train_df['id'] = train_df['id'].astype(str) + '.jpeg'
val_df['id'] = val_df['id'].astype(str) + '.jpeg'

# 4. Data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    featurewise_center=True,
    brightness_range=[0.8, 1.2],
    zoom_range=0.2,
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
print(train_df.head())

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='./data/train_set',
    x_col='id',
    y_col='label',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='./data/train_set',
    x_col='id',
    y_col='label',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=False  # Important for validation data
)

           id label
0  21234.jpeg    14
1  11574.jpeg     0
2  21959.jpeg    16
3  14462.jpeg    11
4  02309.jpeg    13
Found 17944 validated image filenames belonging to 20 classes.
Found 4486 validated image filenames belonging to 20 classes.


In [ ]:
# 5. Build the CNN model
model = Sequential([
    Conv2D(128, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), padding='same'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.5),

    Conv2D(512, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.5),
    
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),

    GlobalAveragePooling2D(),
    Dense(units=512, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

# 6. Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 7. Callbacks
early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=20, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)

model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)

In [8]:
# 8. Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stop, reduce_lr]
)

c:\Users\camil\anaconda3\envs\tf\lib\site-packages\keras\preprocessing\image.py:1863: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


Epoch 1/100
281/281 [==============================] - 71s 244ms/step - loss: 2.3930 - accuracy: 0.2420 - val_loss: 4.6491 - val_accuracy: 0.0633 - lr: 1.0000e-04
Epoch 2/100
281/281 [==============================] - 11s 40ms/step - loss: 1.9759 - accuracy: 0.3485 - val_loss: 2.8621 - val_accuracy: 0.1783 - lr: 1.0000e-04
Epoch 3/100
281/281 [==============================] - 11s 40ms/step - loss: 1.7944 - accuracy: 0.3960 - val_loss: 2.0235 - val_accuracy: 0.3359 - lr: 1.0000e-04
Epoch 4/100
281/281 [==============================] - 11s 40ms/step - loss: 1.6613 - accuracy: 0.4382 - val_loss: 2.2997 - val_accuracy: 0.2976 - lr: 1.0000e-04
Epoch 5/100
281/281 [==============================] - 11s 40ms/step - loss: 1.5459 - accuracy: 0.4765 - val_loss: 2.5854 - val_accuracy: 0.3074 - lr: 1.0000e-04
Epoch 6/100
281/281 [==============================] - 11s 40ms/step - loss: 1.4603 - accuracy: 0.5003 - val_loss: 1.9247 - val_accuracy: 0.4012 - lr: 1.0000e-04
Epoch 7/100
281/281 [======

In [10]:
# 9. Predict on test data
import os
test_filenames = os.listdir('./data/test_set')
test_df = pd.DataFrame({'id': test_filenames})

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='./data/test_set',
    x_col='id',
    y_col = None,
    class_mode=None,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=False  # Important for validation data
    
)

predictions = model.predict(test_generator, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)

Found 4000 validated image filenames.
63/63 [==============================] - 12s 185ms/step


In [11]:
train_labels = train_df['label'].unique()
train_class_mapping = {int(label): idx for idx, label in enumerate(sorted(train_labels))}
print(train_class_mapping)

# Convert predicted class indices to actual labels using the class mapping
predicted_labels = [list(train_class_mapping.keys())[pred] for pred in predicted_classes]

{0: 0, 1: 1, 10: 2, 11: 3, 12: 4, 13: 5, 14: 6, 15: 7, 16: 8, 17: 9, 18: 10, 19: 11, 2: 12, 3: 13, 4: 14, 5: 15, 6: 16, 7: 17, 8: 18, 9: 19}


In [12]:
# 10. Save submission file
submission = pd.DataFrame({'id': test_df['id'], 'label': predicted_labels})
submission['id'] = submission['id'].apply(lambda x: int(os.path.splitext(x)[0]))  # remove .png if necessary
submission.sort_values('id', inplace=True)
submission.to_csv('submission.csv', index=False)